In [10]:
# !pip install feedparser
# !pip install chromadb
# !pip install lxml==4.9.3 lxml_html_clean newspaper3k

In [11]:
import kagglehub
import os
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import hstack
from textblob import TextBlob
from sklearn.pipeline import Pipeline
import re
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.utils import resample
import time
import requests
from bs4 import BeautifulSoup
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import feedparser
from datetime import datetime
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from newspaper import Article
from urllib.parse import urlparse
import spacy
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import textwrap
from sklearn.linear_model import LogisticRegression
import json

In [12]:
path = kagglehub.dataset_download("saketchaturvedi/liarplus")
print("Path to dataset files:", path)

100%|██████████| 3.00M/3.00M [00:00<00:00, 124MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/saketchaturvedi/liarplus/versions/1


In [13]:
dataset_path = os.path.join(path, "LIAR-PLUS-master", "dataset")
print(os.listdir(dataset_path))

['val2.tsv', 'train2.tsv', 'test2.tsv']


In [14]:
df_train = pd.read_csv(os.path.join(dataset_path, "train2.tsv"), sep="\t", header=None)
df_val   = pd.read_csv(os.path.join(dataset_path, "val2.tsv"), sep="\t", header=None)
df_test  = pd.read_csv(os.path.join(dataset_path, "test2.tsv"), sep="\t", header=None)

In [15]:
# names of columns
cols = [
    "index", "id", "label", "statement", "subject", "speaker", "job", "state",
    "party", "barely_true", "false", "half_true", "mostly_true", "pants_on_fire",
    "context", "justification"
]

In [16]:
# assigning appropriate column names to datasets
df_train.columns = cols
df_val.columns   = cols
df_test.columns  = cols

In [17]:
# drop extra index column
df_train = df_train.drop(columns=["index"])
df_val   = df_val.drop(columns=["index"])
df_test  = df_test.drop(columns=["index"])

In [18]:
for df in [df_train, df_val, df_test]:
    df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
    df.reset_index(drop=True, inplace=True)

In [19]:
# remove .json after ID number values
for df in [df_train, df_val, df_test]:
    df["id"] = df["id"].str.replace(".json", "", regex=False)

In [20]:
df_train.head(3)

,id,label,statement,subject,speaker,job,state,party,barely_true,false,half_true,mostly_true,pants_on_fire,context,justification
0,2635,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer,That's a premise that he fails to back up. Ann...
1,10540,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.,"Surovell said the decline of coal ""started whe..."
2,324,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver,Obama said he would have voted against the ame...


# Frequency Heuristic

In [21]:
train_freq = df_train.copy()
val_freq   = df_val.copy()
test_freq  = df_test.copy()

In [22]:
def build_frequency_model(df_train):
    tfidf = TfidfVectorizer(stop_words='english', max_features=5000)
    tfidf_matrix_train = tfidf.fit_transform(df_train['statement'])

    count_vec = CountVectorizer(stop_words='english')
    count_matrix_train = count_vec.fit_transform(df_train['statement'])
    token_freq = np.asarray(count_matrix_train.sum(axis=0)).ravel()
    token_dict = {w: token_freq[i] for i, w in enumerate(count_vec.get_feature_names_out())}

    buzzwords = {'always','never','everyone','nobody','millions','billions','every',
                 'no one','thousands','people say','experts agree'}

    def avg_word_freq(text):
        words = [w for w in text.lower().split() if w in token_dict]
        return np.mean([token_dict[w] for w in words]) if words else 0

    def buzzword_score(text):
        return sum(b in text.lower() for b in buzzwords)

    def repetition_score(text):
        tokens = text.lower().split()
        return 1 - len(set(tokens)) / len(tokens) if tokens else 0

    X_train = pd.DataFrame({
        "tfidf_mean": tfidf_matrix_train.mean(axis=1).A1,
        "word_freq_mean": df_train['statement'].apply(avg_word_freq),
        "buzzword_score": df_train['statement'].apply(buzzword_score),
        "repetition_score": df_train['statement'].apply(repetition_score)
    }).fillna(0)

    le = LabelEncoder()
    y_train = le.fit_transform(df_train['label'])

    model = Pipeline([("scaler", StandardScaler()), ("clf", RandomForestClassifier(n_estimators=200, random_state=42))])
    model.fit(X_train, y_train)

    return model, tfidf, count_vec, token_dict, buzzwords, le

In [23]:
def predict_frequency_model(df, model, tfidf, count_vec, token_dict, buzzwords, le):
    def avg_word_freq(text):
        words = [w for w in text.lower().split() if w in token_dict]
        return np.mean([token_dict[w] for w in words]) if words else 0

    def buzzword_score(text):
        return sum(b in text.lower() for b in buzzwords)

    def repetition_score(text):
        tokens = text.lower().split()
        return 1 - len(set(tokens)) / len(tokens) if tokens else 0

    tfidf_matrix = tfidf.transform(df['statement'])
    X = pd.DataFrame({
        "tfidf_mean": tfidf_matrix.mean(axis=1).A1,
        "word_freq_mean": df['statement'].apply(avg_word_freq),
        "buzzword_score": df['statement'].apply(buzzword_score),
        "repetition_score": df['statement'].apply(repetition_score)
    }).fillna(0)

    preds = model.predict(X)
    probs = model.predict_proba(X).max(axis=1)
    pred_labels = le.inverse_transform(preds)

    label_to_score = {
        "true": 2,
        "mostly-true": 2,
        "half-true": 1,
        "barely-true": 0,
        "false": 0,
        "pants-on-fire": 0
    }

    freq_scores = [label_to_score.get(lbl, 1) for lbl in pred_labels]

    return pd.DataFrame({
        "id": df["id"],
        "statement": df["statement"],
        "predicted_frequency_heuristic": freq_scores,
        "frequency_heuristic_score": probs
    })

In [24]:
model_freq, tfidf_freq, count_vec_freq, token_dict_freq, buzzwords_freq, le_freq = build_frequency_model(train_freq)

In [25]:
val_results = predict_frequency_model(val_freq, model_freq, tfidf_freq, count_vec_freq, token_dict_freq, buzzwords_freq, le_freq)

In [26]:
test_results = predict_frequency_model(test_freq, model_freq, tfidf_freq, count_vec_freq, token_dict_freq, buzzwords_freq, le_freq)

In [27]:
print(val_results.head())

      id                                          statement  \
0  12134  We have less Americans working now than in the...   
1    238  When Obama was sworn into office, he DID NOT u...   
2   7891  Says Having organizations parading as being so...   
3   8169     Says nearly half of Oregons children are poor.   
4    929  On attacks by Republicans that various program...   

   predicted_frequency_heuristic  frequency_heuristic_score  
0                              0                      0.365  
1                              1                      0.260  
2                              2                      0.385  
3                              0                      0.330  
4                              0                      0.255  


# Sensationalism

In [28]:
def build_sensationalism_model(df_train):
    def map_sensationalism_from_counts(row):
        total = row["barely_true"] + row["false"] + row["half_true"] + row["mostly_true"] + row["pants_on_fire"]
        if total == 0:
            return 1
        score = (
            2 * row["barely_true"] +
            3 * row["false"] +
            1 * row["half_true"] +
            0.5 * row["mostly_true"] +
            4 * row["pants_on_fire"]
        ) / total
        if score < 1.5:
            return 0
        elif score < 2.5:
            return 1
        else:
            return 2

    for df in [df_train, df_val, df_test]:
        df["sensationalism"] = df.apply(map_sensationalism_from_counts, axis=1)

    def extract_text_features(text):
        text = str(text)
        exclaim = text.count("!")
        allcaps = len(re.findall(r"\b[A-Z]{2,}\b", text))
        sens_words = sum(1 for w in [
            "shocking","unbelievable","incredible","amazing","outrageous",
            "disaster","terrifying","massive","horrifying","explosive",
            "record-breaking","unprecedented","urgent","worst","best"
        ] if w in text.lower())
        blob = TextBlob(text)
        return exclaim, allcaps, sens_words, abs(blob.sentiment.polarity), blob.sentiment.subjectivity

    feats = df_train["statement"].apply(extract_text_features)
    df_train[["exclaim","allcaps","sens_words","polarity","subjectivity"]] = pd.DataFrame(feats.tolist(), index=df_train.index)

    text_col = "statement"
    meta_features = ["speaker", "party", "context", "job"]
    numeric_features = ["exclaim", "allcaps", "sens_words", "polarity", "subjectivity"]

    preprocessor = ColumnTransformer([
        ("text", TfidfVectorizer(max_features=5000, stop_words="english"), text_col),
        ("cat", OneHotEncoder(handle_unknown="ignore"), meta_features),
        ("num", StandardScaler(), numeric_features)
    ])

    model = XGBClassifier(
        num_class=3,
        n_estimators=300,
        learning_rate=0.1,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        eval_metric="mlogloss"
    )

    pipeline = Pipeline([
        ("preprocess", preprocessor),
        ("model", model)
    ])

    X_train = df_train[[text_col] + meta_features + numeric_features]
    y_train = df_train["sensationalism"]

    pipeline.fit(X_train, y_train)

    return pipeline, preprocessor, meta_features, numeric_features

In [29]:
def predict_sensationalism_model(df, pipeline, preprocessor, meta_features, numeric_features):
    df = df.copy()

    def extract_text_features(text):
        text = str(text)
        exclaim = text.count("!")
        allcaps = len(re.findall(r"\b[A-Z]{2,}\b", text))
        sens_words = sum(1 for w in [
            "shocking","unbelievable","incredible","amazing","outrageous",
            "disaster","terrifying","massive","horrifying","explosive",
            "record-breaking","unprecedented","urgent","worst","best"
        ] if w in text.lower())
        blob = TextBlob(text)
        return exclaim, allcaps, sens_words, abs(blob.sentiment.polarity), blob.sentiment.subjectivity

    feats = df["statement"].apply(extract_text_features)
    df[["exclaim","allcaps","sens_words","polarity","subjectivity"]] = pd.DataFrame(feats.tolist(), index=df.index)

    X = df[["statement"] + meta_features + numeric_features]
    preds = pipeline.predict(X)
    probs = pipeline.predict_proba(X).max(axis=1)

    return pd.DataFrame({
        "id": df["id"],
        "statement": df["statement"],
        "predicted_sensationalism": preds,
        "sensationalism_score": probs
    })

In [30]:
sens_pipeline, sens_preproc, sens_meta, sens_num = build_sensationalism_model(df_train)
val_results_sens = predict_sensationalism_model(df_val, sens_pipeline, sens_preproc, sens_meta, sens_num)

In [31]:
test_results_sens = predict_sensationalism_model(df_test, sens_pipeline, sens_preproc, sens_meta, sens_num)

In [32]:
print(test_results_sens.head())

      id                                          statement  \
0  11972  Building a wall on the U.S.-Mexico border will...   
1  11685  Wisconsin is on pace to double the number of l...   
2  11096  Says John McCain has done nothing to help the ...   
3   5209  Suzanne Bonamici supports a plan that will cut...   
4   9524  When asked by a reporter whether hes at the ce...   

   predicted_sensationalism  sensationalism_score  
0                         1              0.959828  
1                         1              0.478948  
2                         1              0.966644  
3                         1              0.848621  
4                         2              0.821971  


In [33]:
print(val_results_sens.head())

      id                                          statement  \
0  12134  We have less Americans working now than in the...   
1    238  When Obama was sworn into office, he DID NOT u...   
2   7891  Says Having organizations parading as being so...   
3   8169     Says nearly half of Oregons children are poor.   
4    929  On attacks by Republicans that various program...   

   predicted_sensationalism  sensationalism_score  
0                         1              0.669526  
1                         2              0.964778  
2                         0              0.438723  
3                         2              0.371098  
4                         0              0.988581  


# Malicious Account

In [34]:
train_mal = df_train.copy()
val_mal   = df_val.copy()
test_mal  = df_test.copy()

In [35]:
def build_malicious_account_model(df_train):
    tfidf = TfidfVectorizer(stop_words='english', max_features=5000)
    tfidf_matrix_train = tfidf.fit_transform(df_train['statement'])

    def avg_token_length(text):
        tokens = text.split()
        return np.mean([len(t) for t in tokens]) if tokens else 0

    def repetition_score(text):
        tokens = text.lower().split()
        return 1 - len(set(tokens)) / len(tokens) if tokens else 0

    def link_count(text):
        return text.count('http') + text.count('www.')

    def hashtag_mention_count(text):
        return text.count('#') + text.count('@')

    def punctuation_ratio(text):
        punct = sum(1 for c in text if c in "!?.,")
        return punct / len(text) if len(text) > 0 else 0

    def uppercase_ratio(text):
        upper = sum(1 for c in text if c.isupper())
        return upper / len(text) if len(text) > 0 else 0

    X_train = pd.DataFrame({
        "tfidf_mean": tfidf_matrix_train.mean(axis=1).A1,
        "avg_token_length": df_train['statement'].apply(avg_token_length),
        "repetition_score": df_train['statement'].apply(repetition_score),
        "link_count": df_train['statement'].apply(link_count),
        "hashtag_mention_count": df_train['statement'].apply(hashtag_mention_count),
        "punctuation_ratio": df_train['statement'].apply(punctuation_ratio),
        "uppercase_ratio": df_train['statement'].apply(uppercase_ratio)
    }).fillna(0)

    le = LabelEncoder()
    y_train = le.fit_transform(df_train['label'])

    model = Pipeline([
        ("scaler", StandardScaler()),
        ("clf", RandomForestClassifier(n_estimators=200, random_state=42))
    ])
    model.fit(X_train, y_train)

    return model, tfidf, le

In [36]:
def predict_malicious_account_model(df, model, tfidf, le):
    def avg_token_length(text):
        tokens = text.split()
        return np.mean([len(t) for t in tokens]) if tokens else 0

    def repetition_score(text):
        tokens = text.lower().split()
        return 1 - len(set(tokens)) / len(tokens) if tokens else 0

    def link_count(text):
        return text.count('http') + text.count('www.')

    def hashtag_mention_count(text):
        return text.count('#') + text.count('@')

    def punctuation_ratio(text):
        punct = sum(1 for c in text if c in "!?.,")
        return punct / len(text) if len(text) > 0 else 0

    def uppercase_ratio(text):
        upper = sum(1 for c in text if c.isupper())
        return upper / len(text) if len(text) > 0 else 0

    tfidf_matrix = tfidf.transform(df['statement'])
    X = pd.DataFrame({
        "tfidf_mean": tfidf_matrix.mean(axis=1).A1,
        "avg_token_length": df['statement'].apply(avg_token_length),
        "repetition_score": df['statement'].apply(repetition_score),
        "link_count": df['statement'].apply(link_count),
        "hashtag_mention_count": df['statement'].apply(hashtag_mention_count),
        "punctuation_ratio": df['statement'].apply(punctuation_ratio),
        "uppercase_ratio": df['statement'].apply(uppercase_ratio)
    }).fillna(0)

    preds = model.predict(X)
    probs = model.predict_proba(X).max(axis=1)
    pred_labels = le.inverse_transform(preds)

    label_to_score = {
        "true": 2,
        "mostly-true": 2,
        "half-true": 1,
        "barely-true": 0,
        "false": 0,
        "pants-on-fire": 0
    }

    malicious_scores = [label_to_score.get(lbl, 1) for lbl in pred_labels]

    return pd.DataFrame({
        "id": df["id"],
        "statement": df["statement"],
        "predicted_malicious_account": malicious_scores,
        "malicious_account_score": probs
    })

In [37]:
model_malicious, tfidf_malicious, le_malicious = build_malicious_account_model(train_mal)

val_results_malicious = predict_malicious_account_model(val_mal, model_malicious, tfidf_malicious, le_malicious)
test_results_malicious = predict_malicious_account_model(test_mal, model_malicious, tfidf_malicious, le_malicious)

In [38]:
print(val_results_malicious.head())

      id                                          statement  \
0  12134  We have less Americans working now than in the...   
1    238  When Obama was sworn into office, he DID NOT u...   
2   7891  Says Having organizations parading as being so...   
3   8169     Says nearly half of Oregons children are poor.   
4    929  On attacks by Republicans that various program...   

   predicted_malicious_account  malicious_account_score  
0                            1                    0.295  
1                            2                    0.290  
2                            0                    0.360  
3                            0                    0.595  
4                            0                    0.255  


# Naive Realism

In [39]:
def map_naive_realism_from_sentiment(text):

    blob = TextBlob(str(text))
    subj = blob.sentiment.subjectivity
    polarity = abs(blob.sentiment.polarity)
    score = subj + polarity

    if score < 0.4:
        return 0   # balanced / open-minded
    elif score < 0.8:
        return 1   # somewhat naive-realist
    else:
        return 2   # strongly naive-realist

In [40]:
def extract_naive_realism_features(text):

    text = str(text)
    words = text.lower().split()

    cautious_words = ["maybe", "perhaps", "possibly", "likely", "suggests", "could", "might"]
    absolutes = ["always", "never", "everyone", "nobody", "clearly", "undeniably"]
    cautious_ratio = sum(w in cautious_words for w in words) / max(len(words), 1)
    absolute_ratio = sum(w in absolutes for w in words) / max(len(words), 1)

    dismissive_terms = ["idiot", "fool", "biased", "brainwashed", "fake", "delusional"]
    dismissive_count = sum(w in text.lower() for w in dismissive_terms)

    blob = TextBlob(text)
    return (
        absolute_ratio,
        cautious_ratio,
        dismissive_count
    )

In [41]:
def build_naive_realism_model(df_train, df_val, df_test):
    for df in [df_train, df_val, df_test]:
        df["naive_realism"] = df["statement"].apply(map_naive_realism_from_sentiment)

    feats = df_train["statement"].apply(extract_naive_realism_features)
    df_train[["absolute_ratio", "cautious_ratio", "dismissive_count"]] = pd.DataFrame(
        feats.tolist(), index=df_train.index
    )

    text_col = "statement"
    meta_features = ["speaker", "party", "context", "job"]
    numeric_features = ["absolute_ratio", "cautious_ratio", "dismissive_count"]

    preprocessor = ColumnTransformer([
        ("text", TfidfVectorizer(max_features=5000, stop_words="english"), text_col),
        ("cat", OneHotEncoder(handle_unknown="ignore"), meta_features),
        ("num", StandardScaler(), numeric_features)
    ])

    model = XGBClassifier(
        num_class=3,
        n_estimators=300,
        learning_rate=0.1,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        eval_metric="mlogloss"
    )

    pipeline = Pipeline([
        ("preprocess", preprocessor),
        ("model", model)
    ])

    X_train = df_train[[text_col] + meta_features + numeric_features]
    y_train = df_train["naive_realism"]
    pipeline.fit(X_train, y_train)

    return pipeline, preprocessor, meta_features, numeric_features

In [42]:
def predict_naive_realism_model(df, pipeline, preprocessor, meta_features, numeric_features):
    df = df.copy()

    feats = df["statement"].apply(extract_naive_realism_features)
    df[["absolute_ratio", "cautious_ratio", "dismissive_count"]] = pd.DataFrame(
        feats.tolist(), index=df.index
    )

    X = df[["statement"] + meta_features + numeric_features]
    preds = pipeline.predict(X)
    probs = pipeline.predict_proba(X).max(axis=1)

    return pd.DataFrame({
        "id": df["id"],
        "statement": df["statement"],
        "predicted_naive_realism": preds,
        "naive_realism_score": probs
    })

In [43]:
naive_pipeline, naive_preproc, naive_meta, naive_num = build_naive_realism_model(df_train, df_val, df_test)
val_results_naive = predict_naive_realism_model(df_val, naive_pipeline, naive_preproc, naive_meta, naive_num)

In [44]:
print(val_results_naive)

         id                                          statement  \
0     12134  We have less Americans working now than in the...   
1       238  When Obama was sworn into office, he DID NOT u...   
2      7891  Says Having organizations parading as being so...   
3      8169     Says nearly half of Oregons children are poor.   
4       929  On attacks by Republicans that various program...   
...     ...                                                ...   
1279   3419  For the first time in more than a decade, impo...   
1280  12548  Says Donald Trump has bankrupted his companies...   
1281    401  John McCain and George Bush have "absolutely n...   
1282   1055  A new poll shows 62 percent support the presid...   
1283   9117  No one claims the report vindicating New Jerse...   

      predicted_naive_realism  naive_realism_score  
0                           0             0.672541  
1                           0             0.474138  
2                           0             0.6622

# Full Model Pipeline

In [45]:
nlp = spacy.load("en_core_web_sm")

def fetch_article(url):
    article_data = {
        "url": url,
        "title": "",
        "text": "",
        "authors": [],
        "publish_date": None,
        "source": "",
        "keywords": [],
        "error": None
    }
    article = Article(url)
    article.download()
    article.parse()

    article_data["title"] = article.title or ""
    article_data["text"] = article.text or ""
    article_data["authors"] = article.authors or []
    article_data["source"] = urlparse(url).netloc
    article_data["publish_date"] = (
        pd.to_datetime(article.publish_date).date().isoformat()
        if article.publish_date else None
    )

    doc = nlp(article_data["title"] + " " + article_data["text"])
    keywords = [ent.text for ent in doc.ents if ent.label_ in {"ORG", "GPE", "PERSON"}]
    article_data["keywords"] = list(set(keywords))

    return article_data

In [46]:
def train_party_model(df_train):
    party_df = df_train.dropna(subset=["party", "statement"])
    party_le = LabelEncoder()
    y_party = party_le.fit_transform(party_df["party"])

    party_clf = Pipeline([
        ("tfidf", TfidfVectorizer(max_features=5000, stop_words="english")),
        ("logreg", LogisticRegression(max_iter=300, class_weight="balanced"))
    ])
    party_clf.fit(party_df["statement"], y_party)
    return party_clf, party_le

In [47]:
def train_job_model(df_train):
    job_df = df_train.dropna(subset=["job", "statement"])
    job_le = LabelEncoder()
    y_job = job_le.fit_transform(job_df["job"])

    job_clf = Pipeline([
        ("tfidf", TfidfVectorizer(max_features=5000, stop_words="english")),
        ("logreg", LogisticRegression(max_iter=300, class_weight="balanced"))
    ])
    job_clf.fit(job_df["statement"], y_job)
    return job_clf, job_le

In [48]:
def predict_party(text, party_clf, party_le):
    probs = party_clf.predict_proba([text])[0]
    pred_label = party_le.inverse_transform([probs.argmax()])[0]
    return pred_label


def predict_job(text, job_clf, job_le):
    probs = job_clf.predict_proba([text])[0]
    pred_label = job_le.inverse_transform([probs.argmax()])[0]
    return pred_label


def train_party_and_job_models(df_train):
    party_clf, party_le = train_party_model(df_train)
    job_clf, job_le = train_job_model(df_train)
    return (party_clf, party_le), (job_clf, job_le)

(party_clf, party_le), (job_clf, job_le) = train_party_and_job_models(df_train)

def prepare_article_for_models(article):
    text = article.get("text", "").strip()
    title = article.get("title", "")
    source = article.get("source", "")
    authors = article.get("authors", [])
    author = authors[0] if authors else ""
    publish_date = article.get("publish_date", "")
    keywords = article.get("keywords", [])

    doc = nlp(text)
    ents = [ent.text for ent in doc.ents if ent.label_ in {"PERSON","ORG","GPE"}]
    subject = ", ".join(sorted(set(keywords + ents)))[:300]
    t = text.lower()

    job = predict_job(text, job_clf, job_le)
    party = predict_party(text, party_clf, party_le)

    context = f"Article from {source} published {publish_date}"
    is_political = int(any(w in t for w in ["politic", "election", "senate", "president"]))
    subject_length = len(subject.split(",")) if subject else 0

    df = pd.DataFrame([{
        "id": "custom_001",
        "statement": text,
        "subject": subject,
        "speaker": author,
        "job": job,
        "party": party,
        "context": context,
        "subject_length": subject_length,
        "is_political": is_political,
        "source": source,
        "publish_date": publish_date
    }])
    return df

In [49]:
def chunk_text(text, chunk_size=500, overlap=50):
    chunks, start = [], 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start += chunk_size - overlap
    return chunks

In [50]:
def setup_rag_for_article(article_df):
    text = article_df["statement"].iloc[0]
    chunks = chunk_text(text)
    embedder = SentenceTransformer("all-MiniLM-L6-v2")
    embeddings = embedder.encode(chunks)

    client = chromadb.Client(Settings(anonymized_telemetry=False))
    collection = client.get_or_create_collection(name="articles")
    collection.add(
        ids=[f"chunk_{i}" for i in range(len(chunks))],
        documents=chunks,
        embeddings=embeddings.tolist(),
        metadatas=[{
            "source": article_df["source"].iloc[0],
            "publish_date": article_df["publish_date"].iloc[0]
        } for _ in chunks]
    )
    return collection, embedder

In [51]:
def retrieve_context(query, collection, embedder, n_results=3):
    query_embedding = embedder.encode([query])
    results = collection.query(query_embeddings=query_embedding.tolist(), n_results=n_results)
    retrieved = [doc for doc in results["documents"][0]]
    return "\n\n".join(retrieved)

# Setup

Please ensure you have imported a Gemini API key from AI Studio.
You can do this directly in the Secrets tab on the left.

After doing so, please run the setup cell below.

In [55]:
!pip install -U -q "google"
!pip install -U -q "google.genai"

import os
from google.colab import userdata
from google.colab import drive
os.environ["GEMINI_API_KEY"] = userdata.get("capstone")

# drive.mount("/content/drive")
# # Please ensure that uploaded files are available in the AI Studio folder or change the working folder.
# os.chdir("/content/drive/MyDrive/Google AI Studio")

# Generated Code

In [ ]:
cnn_df = prepare_article_for_models(fetch_article("https://www.cnn.com/2025/10/20/politics/trump-no-kings-protests-vance-cia-analysis"))
article_text = cnn_df['statement'][0]

In [ ]:
sens_pred = predict_sensationalism_model(cnn_df, sens_pipeline, sens_preproc, sens_meta, sens_num)['predicted_sensationalism'][0]
nr_pred = predict_naive_realism_model(cnn_df, naive_pipeline, naive_preproc, naive_meta, naive_num)['predicted_naive_realism'][0]
ma_pred = predict_malicious_account_model(cnn_df, model_malicious, tfidf_malicious, le_malicious)['predicted_malicious_account'][0]
freq_pred = predict_frequency_model(cnn_df, model_freq, tfidf_freq, count_vec_freq, token_dict_freq, buzzwords_freq, le_freq)['predicted_frequency_heuristic'][0]

In [ ]:
test = predict_frequency_model(cnn_df, model_freq, tfidf_freq, count_vec_freq, token_dict_freq, buzzwords_freq, le_freq)

In [ ]:
test

,id,statement,predicted_frequency_heuristic,frequency_heuristic_score
0,custom_001,A president steeped in constitutional lore mig...,2,0.29


In [ ]:
prompt = f"""
You are an expert in misinformation and disinformation detection.
Your task is to analyze the given article and score how strongly it exhibits each factuality factor.
---

### Factuality Factors:
1. **Frequency Heuristic**
  - *Repetition Analysis*: Observe how often a claim or narrative is echoed across the text or across references.
  - *Origin Tracing*: Determine whether frequently repeated information is traced to a credible or questionable source.
  - *Evidence Verification*: Evaluate if the text implies truth merely due to repetition or popularity of a claim.
  - **Scoring:** 0 = none/minimal repetition; 1 = moderate repetition or common-belief phrasing; 2 = heavy repetition or appeal to consensus.

2. **Malicious Account**
  - *Account Analysis*: If the text references or cites accounts, consider whether their creation dates or activity patterns suggest inauthentic behavior.
  - *Interaction Patterns*: Evaluate if the content originates from or interacts with accounts resembling coordinated or bot-like behavior.
  - *Content Review*: Assess if the account or source repeatedly spreads false or harmful information.
  - **Scoring:** 0 = credible source; 1 = slightly suspicious or biased source; 2 = clearly deceptive, coordinated, or malicious source.

3. **Sensationalism**
  - *Language Intensity*: Examine the text for overly dramatic or exaggerated claims.
  - *Tone Comparison*: Compare emotional tone of headlines versus main content.
  - *Shock vs. Substance*: Determine whether the article prioritizes shock value over factual reporting.
  - **Scoring:** 0 = neutral/objective; 1 = mildly emotional or dramatic; 2 = highly sensationalized

4. **Naive Realism**
  - *Perspective Analysis*: Evaluate whether the content presents its view as the only correct one.
  - *Dissenting View Checks*: Analyze whether differing views are acknowledged or dismissed.
  - *Isolation Analysis*: Determine whether the article attempts to isolate readers from alternative perspectives.
  - **Scoring:** 0 = balanced and nuanced; 1 = somewhat one-sided; 2 = fully dogmatic.

You are also given reference model scores from predictive models.
Treat these as informative context, not ground truth, and reason independently.

### Reference Model Scores
- Frequency Heuristic: {freq_pred}
- Malicious Account: {ma_pred}
- Sensationalism: {sens_pred}
- Naive Realism: {nr_pred}

### Article to Evaluate:
{article_text}

### Instructions:
1. Think step-by-step about the article’s tone, evidence, framing, and intent.
2. Identify linguistic cues that signal bias, repetition, exaggeration, or malicious intent.
3. Provide a numeric score and a justification for why that score was chosen.
4. Return **only** a valid JSON object containing the score and reasoning for each of the four factors.

### Output Format:
{{
    "frequency_heuristic": {{
        "score": 0|1|2,
        "reasoning": "Explanation"
  }},
    "malicious_account": {{
        "score": 0|1|2,
        "reasoning": "Explanation"
  }},
    "sensationalism": {{
        "score": 0|1|2,
        "reasoning": "Explanation"
  }},
    "naive_realism": {{
        "score": 0|1|2,
        "reasoning": "Explanation"
  }}
}}
"""

In [ ]:
# To run this code you need to install the following dependencies:
# pip install google-genai

import base64
import os
from google import genai
from google.genai import types


In [ ]:
def generate(prompt):
    client = genai.Client(
        api_key=os.environ.get("GEMINI_API_KEY"),
    )

    model = "gemini-2.5-pro"
    contents = [
        types.Content(
            role="user",
            parts=[
                types.Part.from_text(text=prompt),
            ],
        ),
    ]
    tools = [
        types.Tool(googleSearch=types.GoogleSearch(
        )),
    ]
    generate_content_config = types.GenerateContentConfig(
        thinking_config = types.ThinkingConfig(
            thinking_budget=-1,
        ),
        tools=tools,
    )

    for chunk in client.models.generate_content_stream(
        model=model,
        contents=contents,
        config=generate_content_config,
    ):
        print(chunk.text, end="")

if __name__ == "__main__":
    generate(prompt)


```json
{
    "frequency_heuristic": {
        "score": 2,
        "reasoning": "The article heavily and repeatedly uses the central narrative of Trump as a 'king' or 'autocrat'. This theme is consistently echoed throughout the text, framing various events—such as social media posts, a sentence commutation, and foreign policy decisions—as evidence of his 'monarchical' ambitions and 'unchecked power'. Phrases like 'wannabe autocrat', 'pretensions to absolute power', and 'imperial presidency' are used multiple times to reinforce this single interpretation, making it a clear example of heavy repetition to advance a thesis."
  },
    "malicious_account": {
        "score": 0,
        "reasoning": "The article cites credible and verifiable sources. It references established news organizations (CNN, AP, Reuters, Getty Images), quotes public figures from across the political spectrum (Donald Trump, JD Vance, Rand Paul, Nick LaLota), and attributes information to specific individuals and spoke

# Different Prompting

In [ ]:
def export_hand_label_template():
    sample = df_val.sample(20, random_state=42)[
        ["id", "statement", "label", "speaker", "party", "context"]
    ]
    sample.to_csv("hand_labels_template.csv", index=False)
    return sample

In [ ]:
export_hand_label_template()

,id,statement,label,speaker,party,context
1244,8905,On the Cuba embargo,false,charlie-crist,democrat,"an interview on HBO's ""Real Time"""
331,9833,My campaign alone has created more jobs in the...,mostly-true,clay-pell,democrat,a TV debate
1005,1519,"Since 1981, reconciliation has been used 21 ti...",true,harry-reid,democrat,the televised health care summit
411,7928,The state GOP ticket says their top priority i...,false,terry-mcauliffe,democrat,a speech.
1204,4325,The Dodd-Frank financial-reform laws hundreds ...,barely-true,sean-duffy,republican,an op-ed piece in the Washington Times
851,8818,We have the lowest beer tax in the nation.,false,rod-monroe,democrat,an Oregon Senate work session
247,105,"The reality is, with a $2 trillion-a-year heal...",barely-true,mike-huckabee,republican,NaN
858,8847,"For the first time in over a decade, business ...",mostly-true,barack-obama,democrat,State of the Union address
298,731,Sarah Palin endorsed a Wasilla policy that cha...,half-true,blog-posting,none,blog postings on many Web sites.
553,908,"Energy nominee Steven Chu has called coal ""his...",mostly-true,james-inhofe,republican,in a news release


In [ ]:
client = genai.Client(api_key=os.environ.get("GEMINI_API_KEY"))


cnn_df = prepare_article_for_models(
    fetch_article("https://www.cnn.com/2025/10/20/politics/trump-no-kings-protests-vance-cia-analysis")
)
article_text = cnn_df["statement"].iloc[0]

sens_pred = predict_sensationalism_model(
    cnn_df, sens_pipeline, sens_preproc, sens_meta, sens_num
)["predicted_sensationalism"][0]

nr_pred = predict_naive_realism_model(
    cnn_df, naive_pipeline, naive_preproc, naive_meta, naive_num
)["predicted_naive_realism"][0]

ma_pred = predict_malicious_account_model(
    cnn_df, model_malicious, tfidf_malicious, le_malicious
)["predicted_malicious_account"][0]

freq_pred = predict_frequency_model(
    cnn_df, model_freq, tfidf_freq, count_vec_freq, token_dict_freq, buzzwords_freq, le_freq
)["predicted_frequency_heuristic"][0]


def run_prompt(prompt_text, json_mode=False):
    contents = [
        types.Content(
            role="user",
            parts=[types.Part(text=prompt_text)],
        ),
    ]

    config_kwargs = {
        "thinking_config": types.ThinkingConfig(
            thinking_budget=2048,
        ),
    }
    if json_mode:
        config_kwargs["response_mime_type"] = "application/json"

    generate_content_config = types.GenerateContentConfig(**config_kwargs)

    response = client.models.generate_content(
        model="gemini-2.5-pro",
        contents=contents,
        config=generate_content_config,
    )

    print("RAW RESPONSE:\n")
    print(response.text)

    parsed = None
    if json_mode:
        try:
            parsed = json.loads(response.text)
            print("\nParsed JSON keys:", parsed.keys())
        except json.JSONDecodeError as e:
            print("\nJSON parse failed:", e)

    return response.text, parsed

In [ ]:
prompt_baseline = f"""
You are an expert in misinformation and disinformation detection.

Your task is to read the article below and score how strongly it exhibits each of the following factuality factors:

1. Frequency Heuristic (0–2)
   - 0 = little or no repetition of a claim as a shortcut for truth.
   - 1 = some repetition or “people say…” framing.
   - 2 = strong reliance on repetition or popularity to imply truth.

2. Malicious Account (0–2)
   - 0 = credible, professional, or clearly attributed sources.
   - 1 = somewhat suspicious or biased sources, but not clearly malicious.
   - 2 = clearly deceptive, anonymous, or coordinated behavior.

3. Sensationalism (0–2)
   - 0 = neutral, measured tone.
   - 1 = mildly emotional or dramatic, but still grounded.
   - 2 = highly sensationalized, shock-driven language.

4. Naive Realism (0–2)
   - 0 = balanced, acknowledges alternative views.
   - 1 = somewhat one-sided, but not totally closed off.
   - 2 = dogmatic, treats its view as the only rational one.

INSTRUCTIONS:
- Ignore any external models or tools.
- Just use your own reading of the article.
- For each factor:
  - Give a numeric score from 0–2.
  - Provide 2–3 sentences explaining why.

ARTICLE:
{article_text}

OUTPUT FORMAT (plain text is fine for this pass):
- Frequency Heuristic: <score> – <short explanation>
- Malicious Account: <score> – <short explanation>
- Sensationalism: <score> – <short explanation>
- Naive Realism: <score> – <short explanation>
"""

raw_baseline, _ = run_prompt(prompt_baseline, json_mode=False)


RAW RESPONSE:

- Frequency Heuristic: 1 – The article repeatedly uses a cluster of related terms like “king,” “monarchical,” “autocrat,” and “absolute power” to frame every action taken by the fictional Trump administration. While it provides examples to support this frame, the high frequency of these loaded terms functions as a thematic shortcut, reinforcing the central claim through repetition rather than just through argumentation. This creates an impression of established fact through constant association.

- Malicious Account: 2 – The article is presented in the style of a credible news analysis piece but describes a fictional political reality (e.g., JD Vance as Vice President, a government shutdown in a second Trump term) as if it were fact. This is a fundamentally deceptive framing that deliberately blurs the line between speculative fiction and actual news reporting. The lack of an author or a clear disclaimer identifying the piece as hypothetical makes it a clear example of a

In [ ]:
engineering_prompts = []

In [ ]:
# 2.1 – Adds explicit factor headings
engineering_prompts.append(f"""
You are analyzing the same article as before.

For each of the four factuality factors:
1. Frequency Heuristic
2. Malicious Account
3. Sensationalism
4. Naive Realism

Give:
- A short description of how the article behaves along that dimension.
- A score from 0–2.

Be explicit, like:

Factor: Frequency Heuristic
Score: X
Explanation: ...

ARTICLE:
{article_text}
""")

In [ ]:
raw_eng1, parsed_eng1 = run_prompt(engineering_prompts[0], json_mode=False)

RAW RESPONSE:

Here is the analysis of the article based on the four factuality factors.

***

**Factor: Frequency Heuristic**
**Score: 2**
**Explanation:** The article heavily relies on repetition to reinforce its central theme. Words and concepts related to monarchy and authoritarianism are used repeatedly throughout the text, including "king," "monarchical," "autocrat," "unchecked power," "absolute ruler," "imperious," and "imperial presidency." This constant repetition of the core idea across different events (social media posts, pardons, foreign policy) makes the argument that Trump is acting like a king feel more pervasive and credible.

**Factor: Malicious Account**
**Score: 2**
**Explanation:** The article consistently attributes Trump's actions to a malicious desire for power and a fundamental contempt for democratic norms. Phrases like "growing hubris," "striking contempt," and wielding the legal system "to help his friends and hurt his foes" frame his behavior as stemming fr

In [ ]:
# 2.2 – Require explicit rubric reference
engineering_prompts.append(f"""
Using the definitions and scale below, score the article.

Frequency Heuristic (0–2): repetition and appeal to popularity.
Malicious Account (0–2): malicious or coordinated accounts as source.
Sensationalism (0–2): exaggerated or emotional language.
Naive Realism (0–2): presents its view as the only valid one.

For each factor:
- Mention specific phrases or sections of the article when possible.
- Then give a score.

ARTICLE:
{article_text}
""")

In [ ]:
raw_eng2, parsed_eng2 = run_prompt(engineering_prompts[1], json_mode=False)

RAW RESPONSE:

Based on the definitions provided, here is a scoring of the article for each factor.

### **Frequency Heuristic (0–2)**
*   **Analysis:** The article heavily relies on repetition of a central theme and an appeal to the popularity of that theme.
    *   **Repetition:** The idea that Trump is a "king," "monarch," or "autocrat" is repeated consistently throughout the text. Specific phrases include: "acting as a king," "No Kings’ protests," "wannabe autocrat," "divine and absolute ruler," "pretensions to absolute power," "behavior becomes more imperious," "latest monarchical moves," "regal whims," "authoritarian writ," and "imperial presidency." This repetition serves to reinforce the article's central argument.
    *   **Appeal to Popularity:** The article repeatedly cites the large number of protesters as evidence for the validity of its claims. Phrases include: "protests by millions of Americans," "millions of people turning out today," "2,700 events in 50 states," and "O

In [ ]:
# 2.3 – Ask for bullet point evidence
engineering_prompts.append(f"""
For each factor (Frequency Heuristic, Malicious Account, Sensationalism, Naive Realism):

1. List 2–4 bullet points of evidence from the article (paraphrased).
2. Then give a final score 0–2 and a one-sentence reason.

ARTICLE:
{article_text}
""")

In [ ]:
raw_eng3, parsed_eng3 = run_prompt(engineering_prompts[2], json_mode=False)

RAW RESPONSE:

Here are the analyses for each factor based on the provided article.

### **Frequency Heuristic**

*   **Evidence:**
    *   The article repeatedly uses words like “king,” “monarchical,” “autocrat,” and “imperial” to describe the president and his actions.
    *   The central framing of the piece is the “No Kings” protest, which is mentioned at the beginning, middle, and end to reinforce the theme of monarchical behavior.
    *   The concept of Trump having or seeking “unchecked power,” “absolute power,” or being “all-powerful” is mentioned multiple times throughout the text.
*   **Score & Reason:**
    *   **Score: 2**
    *   **Reason:** The article consistently repeats the "king" and "autocrat" theme to frame every action, making this characteristic seem pervasive and central to the political situation.

### **Malicious Account**

*   **Evidence:**
    *   The president’s social media account shared an AI meme depicting him as "KING TRUMP" dumping raw sewage on protes

In [ ]:
# 2.4 – Ask for low / medium / high + numeric mapping
engineering_prompts.append(f"""
Evaluate the article along four factors.

For each factor:
- First classify as "low", "medium", or "high".
- Then map that to 0, 1, or 2.
- Explain why.

Factors:
- Frequency Heuristic
- Malicious Account
- Sensationalism
- Naive Realism

ARTICLE:
{article_text}
""")

In [ ]:
raw_eng4, parsed_eng4 = run_prompt(engineering_prompts[3], json_mode=False)

RAW RESPONSE:

Based on the article provided, here is an evaluation along the four requested factors.

### **Frequency Heuristic**

*   **Classification:** High
*   **Map:** 2
*   **Explanation:** The article relentlessly repeats the central theme that Donald Trump is acting like a king or an autocrat. This idea is introduced in the first sentence and is used as the framing device for every subsequent point. Words and phrases like “king,” “monarchical,” “autocrat,” “unchecked power,” “absolute ruler,” “imperious,” and “authoritarian” are used repeatedly throughout the text. By framing every action—from social media posts to military strikes and presidential pardons—as further proof of this single "Trump as king" thesis, the article makes the concept feel pervasive and highly significant, which is the core effect of the frequency heuristic.

### **Malicious Account**

*   **Classification:** Low
*   **Map:** 0
*   **Explanation:** This factor assesses whether the source of the informati

In [ ]:
# 2.5 – Introduce uncertainty
engineering_prompts.append(f"""
For each factuality factor, provide:
- A 0–2 score.
- A 0–1 confidence value (how certain you are).
- 2–3 sentences explaining the decision.

Factors: Frequency Heuristic, Malicious Account, Sensationalism, Naive Realism.

ARTICLE:
{article_text}
""")

In [ ]:
raw_eng5, parsed_eng5 = run_prompt(engineering_prompts[4], json_mode=False)

RAW RESPONSE:

Here is an analysis of the article for each factuality factor.

### **Frequency Heuristic**

*   **Score:** 2/2
*   **Confidence:** 1.0
*   **Explanation:** The article constructs its fictional narrative by extrapolating from real, frequently repeated criticisms and concerns about Donald Trump. It heavily relies on the audience's existing familiarity with themes like his supposed authoritarian tendencies, use of pardons for allies, and "strongman" rhetoric to make its speculative scenario feel plausible and resonant. This leveraging of established, high-frequency narratives is a clear and effective use of the heuristic to build a persuasive, albeit fictional, argument.

### **Malicious Account**

*   **Score:** 0/2
*   **Confidence:** 0.9
*   **Explanation:** The article is a piece of speculative political commentary, not a factual report from a deceptive source. While it is highly critical, its use of easily verifiable counter-factuals (e.g., JD Vance as Vice President,

In [ ]:
# 2.6 – Ask to separate "language" vs "structure"
engineering_prompts.append(f"""
For each factor, structure your answer like this:

- Language cues: ...
- Structural cues: ...
- Score (0–2): ...

Factors:
- Frequency Heuristic
- Malicious Account
- Sensationalism
- Naive Realism

ARTICLE:
{article_text}
""")

In [ ]:
raw_eng6, parsed_eng6 = run_prompt(engineering_prompts[5], json_mode=False)

RAW RESPONSE:

Here is the analysis of the article for each factor.

### Frequency Heuristic

-   **Language cues:** The article uses words and phrases that suggest a repeated and escalating pattern of behavior, such as "growing hubris," "multiplying actions," "not only... but also," "his latest monarchical moves," "far from his first," and "another example of contempt." This language reinforces the idea that the described actions are not isolated incidents but part of a continuous and worsening trend.
-   **Structural cues:** The article is structured as a list of grievances or proofs of its central thesis. It moves from one example to the next (social media posts, the Santos commutation, demands for indictments, military strikes, press regulations) to create an overwhelming impression of a consistent pattern of authoritarian behavior. By stacking these examples, it makes the "kingly" behavior seem constant and pervasive.
-   **Score (0–2):** 2

### Malicious Account

-   **Language c

In [ ]:
# 2.7 – Ask to list specific quoted snippets
engineering_prompts.append(f"""
For each factor, do:

1. List up to 3 short snippets (quoted or paraphrased) that support your judgment.
2. Then give a 0–2 score and explain.

Factors:
- Frequency Heuristic
- Malicious Account
- Sensationalism
- Naive Realism

ARTICLE:
{article_text}
""")

In [ ]:
raw_eng7, parsed_eng7 = run_prompt(engineering_prompts[6], json_mode=False)

RAW RESPONSE:

Based on the article provided, here is an analysis of each factor.

### Frequency Heuristic

1.  **Snippets:**
    *   The article repeatedly uses words and concepts related to royalty and autocracy to describe Trump: "acting as a king," "No Kings protests," "KING TRUMP," "divine and absolute ruler," "wannabe autocrat," "monarchical moves," "regal whims," "imperial presidency," and "increasingly monarchical leader."
    *   The idea of Trump having "unchecked power" or "absolute power" is mentioned at the beginning ("belief that he has unchecked power") and reinforced later ("pretensions to absolute power").
    *   The administration's "contempt" for dissent and democratic processes is stated multiple times: "betrays striking contempt for tens of millions of Americans," "contempt for the democratic process," and "a response that breathes contempt for the values protesters claim to be protecting."

2.  **Score and Explanation:**
    *   **Score: 2**
    *   The article r

In [ ]:
# 2.8 – Ask to explicitly mention ambiguity
engineering_prompts.append(f"""
For each factor:

1. Briefly describe how the article behaves on that factor.
2. Mention any ambiguities or edge cases.
3. Give a final 0–2 score.

Factors:
- Frequency Heuristic
- Malicious Account
- Sensationalism
- Naive Realism

ARTICLE:
{article_text}
""")

In [ ]:
raw_eng8, parsed_eng8 = run_prompt(engineering_prompts[7], json_mode=False)

RAW RESPONSE:

Here is the analysis of the article for each factor.

### **Frequency Heuristic**

1.  **Description:** The article makes extensive and repeated use of a central theme: Donald Trump's "monarchical" or "king-like" behavior. This concept is the article's core thesis and is reinforced in nearly every paragraph. Phrases like "acting as a king," "No Kings protests," "unchecked power," "divine and absolute ruler," "wannabe autocrat," "pretensions to absolute power," "imperious," "monarchical moves," and "regal whims" are used relentlessly. This high frequency of related terms is designed to make the central claim feel more significant, pervasive, and self-evident to the reader.
2.  **Ambiguities:** The primary ambiguity is whether this is a manipulative heuristic or simply strong, thematic writing for an opinion piece. An analysis article is expected to have a central thesis and bring evidence to support it. However, the sheer density and repetition of the "king" metaphor go b

In [ ]:
# 2.9 – Ask for compact tabular summary in text
engineering_prompts.append(f"""
Summarize your assessment as a small table in plain text, with columns:

Factor | Score (0–2) | One-sentence justification

Then below the table, optionally add a short paragraph explaining the overall framing.

Factors: Frequency Heuristic, Malicious Account, Sensationalism, Naive Realism.

ARTICLE:
{article_text}
""")

In [ ]:
raw_eng9, parsed_eng9 = run_prompt(engineering_prompts[8], json_mode=False)

RAW RESPONSE:

| Factor | Score (0–2) | One-sentence justification |
| :--- | :--- | :--- |
| Frequency Heuristic | 2 | The article lists a rapid succession of different events to create an overwhelming impression of a single, accelerating trend toward authoritarianism. |
| Malicious Account | 1 | While not presenting verifiable falsehoods due to its speculative nature, the piece mixes real figures with a fictional timeline to create a highly damaging and potentially misleading narrative. |
| Sensationalism | 2 | The text uses loaded and dramatic language ("wannabe autocrat," "obliteration of constitutional curbs," "regal whims") to frame political actions in the most alarming way possible. |
| Naive Realism | 2 | The article presents its critical view of the administration as the only rational perspective, framing opposing views and supporters as simply contemptuous, power-hungry, or unreasonable. |

The article is a piece of speculative political analysis, framing a hypothetical near

In [ ]:
# 2.10 – First time requiring JSON
engineering_prompts.append(f"""
You are now required to output a machine-readable JSON object.

Read the article and produce:

{{
  "frequency_heuristic": {{
    "score": 0,
    "reasoning": "..."
  }},
  "malicious_account": {{
    "score": 0,
    "reasoning": "..."
  }},
  "sensationalism": {{
    "score": 0,
    "reasoning": "..."
  }},
  "naive_realism": {{
    "score": 0,
    "reasoning": "..."
  }}
}}

Only return JSON, no extra text.

ARTICLE:
{article_text}
""")

In [ ]:
raw_eng10, parsed_eng10 = run_prompt(engineering_prompts[9], json_mode=True)

RAW RESPONSE:

{
  "frequency_heuristic": {
    "score": 0,
    "reasoning": "The article is a singular piece of political analysis and commentary. It does not argue its points by referencing the volume or frequency of reporting on the subject matter."
  },
  "malicious_account": {
    "score": 0,
    "reasoning": "The article, while presenting a speculative and highly critical political narrative, appears to be a piece of commentary or analysis from a news organization. There are no direct indicators, such as signs of automation, impersonation, or promotion of harmful disinformation, that would suggest it originates from a malicious account."
  },
  "sensationalism": {
    "score": 9,
    "reasoning": "The article consistently employs strong, emotionally charged, and dramatic language to frame the political situation as a crisis. Phrases like 'unchecked power,' 'wannabe autocrat,' 'obliteration of constitutional curbs,' 'pride-induced fall,' 'authoritarian writ,' and 'unhinged rhetori

In [ ]:
# 2.11 – Same JSON, but with explicit internal thinking
engineering_prompts.append(f"""
Internally, think step-by-step about each factor, but do NOT show your chain-of-thought.
Externally, output ONLY the JSON object in the exact schema below:

{{
  "frequency_heuristic": {{
    "score": 0,
    "reasoning": "short explanation"
  }},
  "malicious_account": {{
    "score": 0,
    "reasoning": "short explanation"
  }},
  "sensationalism": {{
    "score": 0,
    "reasoning": "short explanation"
  }},
  "naive_realism": {{
    "score": 0,
    "reasoning": "short explanation"
  }}
}}

ARTICLE:
{article_text}
""")

In [ ]:
raw_eng11, parsed_eng11 = run_prompt(engineering_prompts[10], json_mode=True)

RAW RESPONSE:

{
  "frequency_heuristic": {
    "score": 0,
    "reasoning": "The article is a single, detailed piece of political commentary. It does not show characteristics of being part of a high-frequency, automated, or coordinated campaign. It is a long-form article with specific arguments and citations."
  },
  "malicious_account": {
    "score": 8,
    "reasoning": "The article presents a fictional, future political scenario as if it were a factual news report. It uses real political figures (Donald Trump, JD Vance) but describes events that have not occurred (e.g., a post-2024 Trump presidency, specific protests, and military actions). This framing is intentionally deceptive, blurring the line between speculative commentary and fabricated news, which could mislead readers into believing these hypothetical events are real."
  },
  "sensationalism": {
    "score": 8,
    "reasoning": "The article employs highly sensational and emotionally charged language throughout to construct

In [ ]:
# 2.12 – JSON + hint that model scores exist
engineering_prompts.append(f"""
We will eventually compare your scores to some model outputs,
but for THIS prompt you must ignore any model scores and only use your own judgment.

Internally, think step-by-step. Externally, output ONLY this JSON:

{{
  "frequency_heuristic": {{
    "score": 0,
    "reasoning": "short explanation"
  }},
  "malicious_account": {{
    "score": 0,
    "reasoning": "short explanation"
  }},
  "sensationalism": {{
    "score": 0,
    "reasoning": "short explanation"
  }},
  "naive_realism": {{
    "score": 0,
    "reasoning": "short explanation"
  }}
}}

ARTICLE:
{article_text}
""")

In [ ]:
raw_eng12, parsed_eng12 = run_prompt(engineering_prompts[11], json_mode=True)

RAW RESPONSE:

{
  "frequency_heuristic": {
    "score": 0,
    "reasoning": "The article is a single news analysis piece; the frequency heuristic, which relates to repeated exposure to information, does not apply to a standalone document."
  },
  "malicious_account": {
    "score": 0,
    "reasoning": "The article is from CNN, a mainstream news organization. While it has a clear editorial perspective, it is not a malicious account known for spreading disinformation."
  },
  "sensationalism": {
    "score": 8,
    "reasoning": "The article uses highly emotional and evocative language, such as 'wannabe autocrat,' 'obliteration of constitutional curbs,' and 'regal whims,' to create a sense of alarm and crisis."
  },
  "naive_realism": {
    "score": 7,
    "reasoning": "The article presents its critical interpretation of events as the only objective reality, framing opposing viewpoints as juvenile, contemptuous, or a political trick, without seriously engaging with them."
  }
}

Parsed J

# In Context Learning




In [ ]:
article_link = "https://www.cnn.com/2025/10/20/politics/trump-no-kings-protests-vance-cia-analysis"
cnn_df = prepare_article_for_models(
    fetch_article(article_link)
)
article_text = cnn_df["statement"].iloc[0]

In [ ]:
prompt = f"""
You are an expert in misinformation and disinformation detection.
Your task is to analyze the given article and score how strongly it exhibits each factuality factor.
---

### Factuality Factors:
1. **Frequency Heuristic**
  - *Repetition Analysis*: Observe how often a claim or narrative is echoed across the text or across references.
  - *Origin Tracing*: Determine whether frequently repeated information is traced to a credible or questionable source.
  - *Evidence Verification*: Evaluate if the text implies truth merely due to repetition or popularity of a claim.
  - **Scoring:** 0 = none/minimal repetition; 1 = moderate repetition or common-belief phrasing; 2 = heavy repetition or appeal to consensus.

2. **Malicious Account**
  - *Account Analysis*: If the text references or cites accounts, consider whether their creation dates or activity patterns suggest inauthentic behavior.
  - *Interaction Patterns*: Evaluate if the content originates from or interacts with accounts resembling coordinated or bot-like behavior.
  - *Content Review*: Assess if the account or source repeatedly spreads false or harmful information.
  - **Scoring:** 0 = credible source; 1 = slightly suspicious or biased source; 2 = clearly deceptive, coordinated, or malicious source.

3. **Sensationalism**
  - *Language Intensity*: Examine the text for overly dramatic or exaggerated claims.
  - *Tone Comparison*: Compare emotional tone of headlines versus main content.
  - *Shock vs. Substance*: Determine whether the article prioritizes shock value over factual reporting.
  - **Scoring:** 0 = neutral/objective; 1 = mildly emotional or dramatic; 2 = highly sensationalized

4. **Naive Realism**
  - *Perspective Analysis*: Evaluate whether the content presents its view as the only correct one.
  - *Dissenting View Checks*: Analyze whether differing views are acknowledged or dismissed.
  - *Isolation Analysis*: Determine whether the article attempts to isolate readers from alternative perspectives.
  - **Scoring:** 0 = balanced and nuanced; 1 = somewhat one-sided; 2 = fully dogmatic.

### Article to Evaluate:
{article_text}

### Instructions:
1. Think step-by-step about the article’s tone, evidence, framing, and intent.
2. Identify linguistic cues that signal bias, repetition, exaggeration, or malicious intent.
3. Provide a numeric score and a justification for why that score was chosen.
4. Return **only** a valid JSON object containing the score and reasoning for each of the four factors.

### Output Format:
{{
    "frequency_heuristic": {{
        "score": 0|1|2,
        "reasoning": "Explanation"
  }},
    "malicious_account": {{
        "score": 0|1|2,
        "reasoning": "Explanation"
  }},
    "sensationalism": {{
        "score": 0|1|2,
        "reasoning": "Explanation"
  }},
    "naive_realism": {{
        "score": 0|1|2,
        "reasoning": "Explanation"
  }}
}}
"""

In [ ]:
generate(prompt)

```json
{
    "frequency_heuristic": {
        "score": 1,
        "reasoning": "The article repeats the central claim that Donald Trump is acting like a 'king' and exhibiting 'monarchical' behavior. This narrative is echoed throughout the text, referencing his social media posts, his commutation of George Santos's sentence, and his foreign policy actions. While the claim is repeated, it is linked to specific events and actions, rather than being a standalone assertion repeated without context. The phrasing also leans on the 'No Kings' protests as a recurring motif."
    },
    "malicious_account": {
        "score": 1,
        "reasoning": "The article is an opinion piece from CNN, an organization that multiple sources identify as having a 'Lean Left' bias. While the author, Stephen Collinson, is a veteran journalist, the framing of the article is clearly critical of Donald Trump and his administration. The sourcing within the article primarily consists of quoting political figures (b

In [ ]:
prompt_icl = f"""
You are an expert in misinformation and disinformation detection, scoring, and ranking.
Your task is to analyze the given article and score how strongly it exhibits each factuality factor.
---

### Factuality Factors:
1. **Frequency Heuristic**
  - *Repetition Analysis*: Observe how often a claim or narrative is echoed across the text or across references.
  - *Origin Tracing*: Determine whether frequently repeated information is traced to a credible or questionable source.
  - *Evidence Verification*: Evaluate if the text implies truth merely due to repetition or popularity of a claim.
  - **Scoring:** 0 = none/minimal repetition; 1 = moderate repetition or common-belief phrasing; 2 = heavy repetition or appeal to consensus.

2. **Malicious Account**
  - *Account Analysis*: If the text references or cites accounts, consider whether their creation dates or activity patterns suggest inauthentic behavior.
  - *Interaction Patterns*: Evaluate if the content originates from or interacts with accounts resembling coordinated or bot-like behavior.
  - *Content Review*: Assess if the account or source repeatedly spreads false or harmful information.
  - **Scoring:** 0 = credible source; 1 = slightly suspicious or biased source; 2 = clearly deceptive, coordinated, or malicious source.

3. **Sensationalism**
  - *Language Intensity*: Examine the text for overly dramatic or exaggerated claims.
  - *Tone Comparison*: Compare emotional tone of headlines versus main content.
  - *Shock vs. Substance*: Determine whether the article prioritizes shock value over factual reporting.
  - **Scoring:** 0 = neutral/objective; 1 = mildly emotional or dramatic; 2 = highly sensationalized

4. **Naive Realism**
  - *Perspective Analysis*: Evaluate whether the content presents its view as the only correct one.
  - *Dissenting View Checks*: Analyze whether differing views are acknowledged or dismissed.
  - *Isolation Analysis*: Determine whether the article attempts to isolate readers from alternative perspectives.
  - **Scoring:** 0 = balanced and nuanced; 1 = somewhat one-sided; 2 = fully dogmatic.

### Examples

**Example 1:**
Article: The Federal Reserve announced a 0.25% interest rate increase following its quarterly meeting. Economists had predicted the move based on recent inflation data.
Frequency Heuristic: 0 — Single statement of fact with proper attribution and no repetitive framing.
Malicious Account: 0 — Credible financial news source citing official Federal Reserve announcement.
Sensationalism: 0 — Neutral, technical language appropriate for economic reporting.
Naive Realism: 0 — Presents information objectively without claiming singular truth.

**Example 2:**
Article: WAKE UP PEOPLE! Everyone knows the government is controlling us through 5G towers! Thousands are saying it online. The mainstream media won't tell you the TRUTH because they're in on it!
Frequency Heuristic: 2 — Heavy appeals to popularity ('everyone knows,' 'thousands are saying') to establish credibility through repetition.
Malicious Account: 2 — Anonymous source spreading debunked conspiracy theories.
Sensationalism: 2 — All-caps words, exclamation marks, fear-mongering language designed to provoke emotional response.
Naive Realism: 2 — Presents conspiracy as absolute truth, dismisses all mainstream sources, and attempts to isolate readers.

**Example 3:**
Article: Is your food secretly KILLING you? Experts reveal the hidden dangers lurking in your kitchen. You won't believe what we found!
Frequency Heuristic: 1 — Uses populist framing ('hidden dangers') which suggests knowledge.
Malicious Account: 1 — Clickbait journalism from established outlet but not malicious.
Sensationalism: 2 — Clickbait with dramatic language and mystery ('you won't believe').
Naive Realism: 1 — Implies hidden truth without presenting the actual complexity of food safety.

**Example 4:**
Article: Tech company announces its AI will revolutionize healthcare and transform diagnosis forever. The CEO called it a 'game-changing breakthrough' during yesterday's product launch.
Frequency Heuristic: 1 — Repeats transformative framing.
Malicious Account: 0 — Corporate PR from legitimate company so promotional but not malicious.
Sensationalism: 2 — Extreme claims ('revolutionize,' 'transform forever,' 'game-changing').
Naive Realism: 1 — Presents corporate claims as reality without acknowledging uncertainty or limitations.

**Example 5:**
Article: According to multiple sources familiar with the matter, the investigation is ongoing. Officials declined to comment on specifics, citing the active nature of the case.
Frequency Heuristic: 0 — Reports lack of information honestly without speculation.
Malicious Account: 0 — Standard practice of protecting sources while noting limitations.
Sensationalism: 0 — Neutral tone and restrained language.
Naive Realism: 0 — Explicitly communicates uncertainty and ongoing nature of situation.

**Example 6:**
Article: 'Many people are saying the new policy is unfair. It's becoming clear that something needs to change. More and more citizens are questioning the decision every day.'
Frequency Heuristic: 2 — Repeating claims of rising support that lack clear attribution.
Malicious Account: 1 — Lacks verification.
Sensationalism: 1 — Emotionally charged framing ('unfair') and builds urgency but not extreme language.
Naive Realism: 1 — Implies growing consensus makes position correct without presenting other arguments.

**Example 7:**
Article: The new immigration bill sparked heated debate. Republican lawmakers argue it strengthens border security, while Democratic representatives contend it lacks humanitarian protections. Legal experts are divided on constitutional questions.
Frequency Heuristic: 0 — Multiple perspective stated once with attribution to distinct groups.
Malicious Account: 0 — Mainstream news outlet citing political figures and experts.
Sensationalism: 1 — Word 'heated' adds mild emotional tone but otherwise is balanced.
Naive Realism: 0 — Acknowledges multiple perspectives across parties and expert opinion.

**Example 8:**
Article: Shocking allegations have emerged against the senator. Anonymous sources describe a pattern of misconduct, though the senator's team calls the claims 'politically motivated lies' and notes no formal charges have been filed.
Frequency Heuristic: 1 — 'Pattern of misconduct' suggests multiple mentions.
Malicious Account: 1 — Anonymous sourcing.
Sensationalism: 2 — 'Shocking allegations' is designed to attract attention.
Naive Realism: 0 — Presents both accusation and denial. Also acknowledges lack of formal charges.

**Example 9:**
Article: As commonly understood by most experts in the field, renewable energy will replace fossil fuels within two decades. Some outlier researchers dispute this timeline, but the consensus is clear.
Frequency Heuristic: 1 — Frames majority view as established fact.
Malicious Account: 0 — Appears to be legitimate climate journalism citing expert community.
Sensationalism: 0 — Factual tone without emotional manipulation or exaggeration.
Naive Realism: 2 — Dismisses dissenting views and presents prediction as inevitable.

**Example 10:**
Article: Influencer @TrendyLifestyle123 claims this detox tea cured her chronic illness. The product has 50K followers and hundreds of testimonials, though medical professionals warn these supplements are unregulated.
Frequency Heuristic: 1 — Implies truth through follower count and testimonials, but includes medical warning.
Malicious Account: 2 — Suspicious account name and health claims.
Sensationalism: 1 — 'Cured chronic illness' is dramatic claim but is presented as a quote from influencer.
Naive Realism: 1 — Amplifies an unverified claim but includes expert skepticism.

### Article to Evaluate:
{article_text}


### Instructions:
1. Think step-by-step about the article’s tone, evidence, framing, and intent.
2. Identify linguistic cues that signal bias, repetition, exaggeration, or malicious intent.
3. Provide a numeric score and a justification for why that score was chosen.
4. Return **only** a valid JSON object containing the score and reasoning for each of the four factors.

### Output Format:
{{
    "frequency_heuristic": {{
        "score": 0|1|2,
        "reasoning": "Explanation"
  }},
    "malicious_account": {{
        "score": 0|1|2,
        "reasoning": "Explanation"
  }},
    "sensationalism": {{
        "score": 0|1|2,
        "reasoning": "Explanation"
  }},
    "naive_realism": {{
        "score": 0|1|2,
        "reasoning": "Explanation"
  }}
}}
"""

In [ ]:
generate(prompt_icl)

```json
{
    "frequency_heuristic": {
        "score": 1,
        "reasoning": "The article repeatedly frames Donald Trump's actions around the central narrative of him acting like a 'king' or an 'autocrat.' Phrases like 'unlocked power,' 'wannabe autocrat,' 'all-powerful,' 'pretensions to absolute power,' and 'monarchical moves' are used multiple times to reinforce this theme. It also references 'multiplying actions' and 'growing numbers of Americans' to suggest an increasing trend, implying truth through momentum rather than presenting isolated facts."
    },
    "malicious_account": {
        "score": 1,
        "reasoning": "The article is written from a clear anti-Trump perspective, which introduces a strong bias. While it attributes quotes to named individuals (e.g., Karoline Leavitt, Rand Paul, Colleen Connell), the overall framing and sourcing are selective to support its central thesis. The sourcing is not overtly malicious or fabricated, but it is one-sided. For example, the

# Predictive Model Function Calling

In [ ]:
article_link = "https://www.cnn.com/2025/10/20/politics/trump-no-kings-protests-vance-cia-analysis"
cnn_df = prepare_article_for_models(
    fetch_article(article_link)
)
article_text = cnn_df["statement"].iloc[0]

NameError: name 'prepare_article_for_models' is not defined

In [ ]:
prompt_tool = f"""
You are an expert in misinformation and disinformation detection.
Your task is to analyze the given article and score how strongly it exhibits each factuality factor.
---

### Factuality Factors:
1. **Frequency Heuristic**
  - *Repetition Analysis*: Observe how often a claim or narrative is echoed across the text or across references.
  - *Origin Tracing*: Determine whether frequently repeated information is traced to a credible or questionable source.
  - *Evidence Verification*: Evaluate if the text implies truth merely due to repetition or popularity of a claim.
  - **Scoring:** 0 = none/minimal repetition; 1 = moderate repetition or common-belief phrasing; 2 = heavy repetition or appeal to consensus.

2. **Malicious Account**
  - *Account Analysis*: If the text references or cites accounts, consider whether their creation dates or activity patterns suggest inauthentic behavior.
  - *Interaction Patterns*: Evaluate if the content originates from or interacts with accounts resembling coordinated or bot-like behavior.
  - *Content Review*: Assess if the account or source repeatedly spreads false or harmful information.
  - **Scoring:** 0 = credible source; 1 = slightly suspicious or biased source; 2 = clearly deceptive, coordinated, or malicious source.

3. **Sensationalism**
  - *Language Intensity*: Examine the text for overly dramatic or exaggerated claims.
  - *Tone Comparison*: Compare emotional tone of headlines versus main content.
  - *Shock vs. Substance*: Determine whether the article prioritizes shock value over factual reporting.
  - **Scoring:** 0 = neutral/objective; 1 = mildly emotional or dramatic; 2 = highly sensationalized

4. **Naive Realism**
  - *Perspective Analysis*: Evaluate whether the content presents its view as the only correct one.
  - *Dissenting View Checks*: Analyze whether differing views are acknowledged or dismissed.
  - *Isolation Analysis*: Determine whether the article attempts to isolate readers from alternative perspectives.
  - **Scoring:** 0 = balanced and nuanced; 1 = somewhat one-sided; 2 = fully dogmatic.

### Examples
**Example 1:**
Article: The Federal Reserve announced a 0.25% interest rate increase following its quarterly meeting. Economists had predicted the move based on recent inflation data.
Frequency Heuristic: 0 — Single statement of fact with proper attribution and no repetitive framing.
Malicious Account: 0 — Credible financial news source citing official Federal Reserve announcement.
Sensationalism: 0 — Neutral, technical language appropriate for economic reporting.
Naive Realism: 0 — Presents information objectively without claiming singular truth.

**Example 2:**
Article: WAKE UP PEOPLE! Everyone knows the government is controlling us through 5G towers! Thousands are saying it online. The mainstream media won't tell you the TRUTH because they're in on it!
Frequency Heuristic: 2 — Heavy appeals to popularity ('everyone knows,' 'thousands are saying') to establish credibility through repetition.
Malicious Account: 2 — Anonymous source spreading debunked conspiracy theories.
Sensationalism: 2 — All-caps words, exclamation marks, fear-mongering language designed to provoke emotional response.
Naive Realism: 2 — Presents conspiracy as absolute truth, dismisses all mainstream sources, and attempts to isolate readers.

**Example 3:**
Article: Is your food secretly KILLING you? Experts reveal the hidden dangers lurking in your kitchen. You won't believe what we found!
Frequency Heuristic: 1 — Uses populist framing ('hidden dangers') which suggests knowledge.
Malicious Account: 1 — Clickbait journalism from established outlet but not malicious.
Sensationalism: 2 — Clickbait with dramatic language and mystery ('you won't believe').
Naive Realism: 1 — Implies hidden truth without presenting the actual complexity of food safety.

**Example 4:**
Article: Tech company announces its AI will revolutionize healthcare and transform diagnosis forever. The CEO called it a 'game-changing breakthrough' during yesterday's product launch.
Frequency Heuristic: 1 — Repeats transformative framing.
Malicious Account: 0 — Corporate PR from legitimate company so promotional but not malicious.
Sensationalism: 2 — Extreme claims ('revolutionize,' 'transform forever,' 'game-changing').
Naive Realism: 1 — Presents corporate claims as reality without acknowledging uncertainty or limitations.

**Example 5:**
Article: According to multiple sources familiar with the matter, the investigation is ongoing. Officials declined to comment on specifics, citing the active nature of the case.
Frequency Heuristic: 0 — Reports lack of information honestly without speculation.
Malicious Account: 0 — Standard practice of protecting sources while noting limitations.
Sensationalism: 0 — Neutral tone and restrained language.
Naive Realism: 0 — Explicitly communicates uncertainty and ongoing nature of situation.

**Example 6:**
Article: 'Many people are saying the new policy is unfair. It's becoming clear that something needs to change. More and more citizens are questioning the decision every day.'
Frequency Heuristic: 2 — Repeating claims of rising support that lack clear attribution.
Malicious Account: 1 — Lacks verification.
Sensationalism: 1 — Emotionally charged framing ('unfair') and builds urgency but not extreme language.
Naive Realism: 1 — Implies growing consensus makes position correct without presenting other arguments.

**Example 7:**
Article: The new immigration bill sparked heated debate. Republican lawmakers argue it strengthens border security, while Democratic representatives contend it lacks humanitarian protections. Legal experts are divided on constitutional questions.
Frequency Heuristic: 0 — Multiple perspective stated once with attribution to distinct groups.
Malicious Account: 0 — Mainstream news outlet citing political figures and experts.
Sensationalism: 1 — Word 'heated' adds mild emotional tone but otherwise is balanced.
Naive Realism: 0 — Acknowledges multiple perspectives across parties and expert opinion.

**Example 8:**
Article: Shocking allegations have emerged against the senator. Anonymous sources describe a pattern of misconduct, though the senator's team calls the claims 'politically motivated lies' and notes no formal charges have been filed.
Frequency Heuristic: 1 — 'Pattern of misconduct' suggests multiple mentions.
Malicious Account: 1 — Anonymous sourcing.
Sensationalism: 2 — 'Shocking allegations' is designed to attract attention.
Naive Realism: 0 — Presents both accusation and denial. Also acknowledges lack of formal charges.


**Example 9:**
Article: As commonly understood by most experts in the field, renewable energy will replace fossil fuels within two decades. Some outlier researchers dispute this timeline, but the consensus is clear.
Frequency Heuristic: 1 — Frames majority view as established fact.
Malicious Account: 0 — Appears to be legitimate climate journalism citing expert community.
Sensationalism: 0 — Factual tone without emotional manipulation or exaggeration.
Naive Realism: 2 — Dismisses dissenting views and presents prediction as inevitable.

**Example 10:**
Article: Influencer @TrendyLifestyle123 claims this detox tea cured her chronic illness. The product has 50K followers and hundreds of testimonials, though medical professionals warn these supplements are unregulated.
Frequency Heuristic: 1 — Implies truth through follower count and testimonials, but includes medical warning.
Malicious Account: 2 — Suspicious account name and health claims.
Sensationalism: 1 — 'Cured chronic illness' is dramatic claim but is presented as a quote from influencer.
Naive Realism: 1 — Amplifies an unverified claim but includes expert skepticism.

### Article to Evaluate:
{article_text}

### Tool Usage Requirement:
You also have access to tools that return a predictive model score for each factuality factor.
Treat these model scores as informative context, NOT ground truth. You must reason independently.
Before generating your final JSON answer, you must call all four tools using the article_url provided.
Use this URL when calling the tools: {article_link}

### Instructions:
1. Think step-by-step about the article’s tone, evidence, framing, and intent.
2. Identify linguistic cues that signal bias, repetition, exaggeration, or malicious intent.
3. Use your independent analysis of the article and the tool outputs to provide a numeric score and a justification for why that score was chosen. If your score is different than the predictive model score, you must explain why you disagree.
4. Return **only** a valid JSON object containing the score and reasoning for each of the four factors.

### Output Format:
{{
    "frequency_heuristic": {{
        "score": 0|1|2,
        "reasoning": "Explanation"
  }},
    "malicious_account": {{
        "score": 0|1|2,
        "reasoning": "Explanation"
  }},
    "sensationalism": {{
        "score": 0|1|2,
        "reasoning": "Explanation"
  }},
    "naive_realism": {{
        "score": 0|1|2,
        "reasoning": "Explanation"
  }}
}}
"""

In [ ]:
def run_frequency_model(article_url: str):
    """
    Given an article URL, this tool computes a score for the level of
    frequency heuristic exhibited in the article (0-2)

    Args:
      article_url : The URL of the article to evaluate.

    Returns:
      dict with score 0, 1, or 2

    """
    article = fetch_article(article_url)
    df = prepare_article_for_models(article)

    out = predict_frequency_model(
        df, model_freq, tfidf_freq, count_vec_freq,
        token_dict_freq, buzzwords_freq, le_freq
    )

    score = int(out["predicted_frequency_heuristic"].iloc[0])
    return {"frequency_heuristic": score}


In [ ]:
def run_sensationalism_model(article_url: str):
    """
    Given an article URL, this tool computes sensationalism level (0–2).

    Args:
      article_url : The URL of the article to evaluate.

    Returns:
      dict with score 0, 1, or 2
    """
    article = fetch_article(article_url)
    df = prepare_article_for_models(article)

    out = predict_sensationalism_model(
        df, sens_pipeline, sens_preproc, sens_meta, sens_num
    )

    score = int(out["predicted_sensationalism"].iloc[0])
    return {"sensationalism": score}


In [ ]:
def run_malicious_account_model(article_url: str):
    """
    Given an article url, this tool computes a score for the level of
    malicious account this article exhibits (0-2).

    Args:
      article_url : The URL of the article to evaluate.

    Returns:
      dict with score 0, 1, or 2
    """
    article = fetch_article(article_url)
    df = prepare_article_for_models(article)

    out = predict_malicious_account_model(
        df, model_malicious, tfidf_malicious, le_malicious
    )

    score = int(out["predicted_malicious_account"].iloc[0])
    return {"malicious_account": score}


In [ ]:
def run_naive_realism_model(article_url: str):
    """
    Given an article, this tool scores the degree to which the article presents
    its viewpoint as the only rational truth (0–2).

    Args:
      article_url : The URL of the article to evaluate.

    Returns:
      dict with score 0, 1, or 2
    """
    article = fetch_article(article_url)
    df = prepare_article_for_models(article)

    out = predict_naive_realism_model(
        df, naive_pipeline, naive_preproc, naive_meta, naive_num
    )

    score = int(out["predicted_naive_realism"].iloc[0])
    return {"naive_realism": score}


In [ ]:
function_map = {
    "run_frequency_model": run_frequency_model,
    "run_sensationalism_model": run_sensationalism_model,
    "run_malicious_account_model": run_malicious_account_model,
    "run_naive_realism_model": run_naive_realism_model,
}


In [ ]:
def generate_with_tools(prompt, article_url, max_turns=5):
    client = genai.Client(
        api_key=os.environ.get("GEMINI_API_KEY"),
    )
    tools = [
        types.Tool(
            function_declarations=[
                types.FunctionDeclaration(
                    name="run_frequency_model",
                    description="Returns frequency heuristic score (0-2) for an article.",
                    parameters={
                        "type": "object",
                        "properties": {
                            "article_url": {"type": "string", "description": "URL of article to analyze"}
                        },
                        "required": ["article_url"]
                    }
                ),
                types.FunctionDeclaration(
                    name="run_sensationalism_model",
                    description="Returns sensationalism score (0-2) for an article.",
                    parameters={
                        "type": "object",
                        "properties": {
                            "article_url": {"type": "string", "description": "URL of article to analyze"}
                        },
                        "required": ["article_url"]
                    }
                ),
                types.FunctionDeclaration(
                    name="run_malicious_account_model",
                    description="Returns malicious account score (0-2) for an article.",
                    parameters={
                        "type": "object",
                        "properties": {
                            "article_url": {"type": "string", "description": "URL of article to analyze"}
                        },
                        "required": ["article_url"]
                    }
                ),
                types.FunctionDeclaration(
                    name="run_naive_realism_model",
                    description="Returns naive realism score (0-2) for an article.",
                    parameters={
                        "type": "object",
                        "properties": {
                            "article_url": {"type": "string", "description": "URL of article to analyze"}
                        },
                        "required": ["article_url"]
                    }
                )
            ]
        )
    ]

    contents = [
        types.Content(
            role="user",
            parts=[types.Part.from_text(text=prompt)]
        )
    ]

    for turn in range(max_turns):
        response = client.models.generate_content(
            model="gemini-2.5-pro",
            contents=contents,
            config=types.GenerateContentConfig(
                tools=tools,
                tool_config=types.ToolConfig(
                    function_calling_config=types.FunctionCallingConfig(
                        mode="AUTO"
                    )
                )
            )
        )
        if not response.candidates:
            break

        candidate = response.candidates[0]
        if not candidate.content or not candidate.content.parts:
            break

        function_calls = []
        text_parts = []

        for part in candidate.content.parts:
            if part.function_call is not None:
                function_calls.append(part.function_call)
            if part.text is not None and part.text.strip():
                text_parts.append(part.text)

        if function_calls:
            contents.append(candidate.content)
            tool_response_parts = []
            for fc in function_calls:
                args = dict(fc.args)
                try:
                    result = function_map[fc.name](**args)
                    tool_response_parts.append(
                        types.Part.from_function_response(
                            name=fc.name,
                            response=result
                        )
                    )
                except Exception as e:
                    import traceback
                    traceback.print_exc()
                    tool_response_parts.append(
                        types.Part.from_function_response(
                            name=fc.name,
                            response={"error": str(e)}
                        )
                    )

            contents.append(
                types.Content(
                    role="tool",
                    parts=tool_response_parts
                )
            )
            continue

        if text_parts:
            final_text = "".join(text_parts)
            print(final_text)
            return final_text
        break

result = generate_with_tools(prompt_tool, article_link)

{
    "frequency_heuristic": {
        "score": 2,
        "reasoning": "The article consistently repeats the central theme of the president acting as a 'king' or 'autocrat.' This narrative is reinforced through various phrases like 'unchecked power,' 'wannabe autocrat,' 'pretensions to absolute power,' 'monarchical moves,' and 'imperial presidency.' This heavy repetition of a single interpretive frame, rather than just stating facts, serves to solidify the article's thesis in the reader's mind. The model's score of 2 aligns with this assessment, as the article heavily relies on this recurring framing to build its argument."
    },
    "malicious_account": {
        "score": 0,
        "reasoning": "The article is published by CNN, a well-established, mainstream news organization, not a malicious or inauthentic source. It properly attributes its information to specific sources, including social media posts, public statements from politicians (Rand Paul, Mike Johnson), and interviews wi

# Collected Ground Truth

In [ ]:
ground_truth = pd.DataFrame(columns = ['statement', 'frequency_heuristic', 'malicious_account', 'sensationalism', 'naive_realism'])

In [ ]:
def article_ground_truth(article_link, freq_score, ma_score, sens_score, nr_score, df):
    article_df = prepare_article_for_models(fetch_article(article_link))

    article_text = article_df['statement'][0]

    new_row = {
        'statement': article_text,
        'frequency_heuristic': freq_score,
        'malicious_account': ma_score,
        'sensationalism': sens_score,
        'naive_realism': nr_score,
        'link': article_link
    }

    df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
    return df

In [ ]:
cnn_link = 'https://www.cnn.com/2025/10/20/politics/trump-no-kings-protests-vance-cia-analysis'
ground_truth = article_ground_truth(cnn_link, 2,0,2,2, ground_truth)

In [ ]:
abc7_link = 'https://abc7.com/post/trump-is-committed-giving-americans-2000-tariff-checks-white-house/18149579/'
ground_truth = article_ground_truth(abc7_link,0,0,0,1, ground_truth)

In [ ]:
guardian_link = 'https://www.theguardian.com/us-news/2025/nov/12/house-bill-government-shutdown-vote'
ground_truth = article_ground_truth(guardian_link, 0,0,0,1,ground_truth)

In [ ]:
apn_link = 'https://apnews.com/article/redistricting-gerrymandering-congress-trump-0af8561b1670032fae3e1d2aec7905f0'
ground_truth = article_ground_truth(apn_link, 0,0,0,0,ground_truth)

In [ ]:
act_link = 'https://azcapitoltimes.com/news/2025/11/12/prop-50-exposes-deepening-political-and-geographical-shifts-in-california/'
ground_truth = article_ground_truth(act_link,1,0,1,0,ground_truth)

In [ ]:
abc7_epstein_files = 'https://abcnews.go.com/Politics/house-vote-full-epstein-files-release-move-speaker/story?id=127593181'
ground_truth = article_ground_truth(abc7_epstein_files,0,0,1,1,ground_truth)

In [ ]:
nbc_link = 'https://www.nbcnews.com/politics/politics-news/poll-majorities-parties-extreme-rhetoric-charlie-kirk-killing-rcna243560'
ground_truth = article_ground_truth(nbc_link,0,0,0,0,ground_truth)

In [ ]:
bbc_link = 'https://www.bbc.com/news/articles/c4gpnz05kr8o'
ground_truth = article_ground_truth(bbc_link,1,0,0,0,ground_truth)

In [ ]:
cal_matters = 'https://calmatters.org/commentary/2025/11/dana-williamsons-indictment-political-operatives/'
ground_truth = article_ground_truth(cal_matters,0,0,1,1,ground_truth)

In [ ]:
kff_link = 'https://www.kff.org/racial-equity-and-health-policy/immigrants-report-rising-fear-negative-economic-and-health-impacts-and-changing-political-views-during-the-first-year-of-president-trumps-second-term/'
ground_truth = article_ground_truth(kff_link,0,0,2,0,ground_truth)

In [ ]:
nyt_link = 'https://www.nytimes.com/2025/11/19/opinion/trump-china-xi-trade.html'
ground_truth = article_ground_truth(nyt_link,0,0,1,2,ground_truth)

In [ ]:
dailywire_link = 'https://www.dailywire.com/news/musk-reveals-shocking-cases-of-fraud-found-at-small-business-administration?author=Daily+Wire+News&category=News&elementPosition=25&row=0&rowType=Vertical+List&title=Musk+Reveals+Shocking+Cases+Of+Fraud+Found+At+Small+Business+Administration'
ground_truth = article_ground_truth(dailywire_link,1,0,2,2,ground_truth)

In [ ]:
dppa_link = 'https://dppa.un.org/en/mtg-sc-10027-asg-pobee-30-oct-2025'
ground_truth = article_ground_truth(dppa_link,1,0,1,1,ground_truth)

In [ ]:
cbs_link = 'https://www.cbsnews.com/news/epstein-accusers-lawmakers-vote-epstein-files/'
ground_truth = article_ground_truth(cbs_link,0,0,0,0,ground_truth)

In [ ]:
dailymail_link = 'https://www.dailymail.co.uk/sport/mlb/article-15303149/mark-teixeira-ice-agents-protection-texas.html'
ground_truth = article_ground_truth(dailymail_link,1,0,1,1,ground_truth)

In [ ]:
rollingstone_link = 'https://www.rollingstone.com/politics/politics-news/republicans-healthcare-plan-1235458309/'
ground_truth = article_ground_truth(rollingstone_link,1,0,1,2,ground_truth)

In [ ]:
vanityfair_link = 'https://www.vanityfair.com/news/story/gavin-newsom-presidential-campaign-2028'
ground_truth = article_ground_truth(vanityfair_link,1,0,1,1,ground_truth)

In [ ]:
mpr_link = 'https://www.mprnews.org/story/2025/11/18/minnesota-gun-law-vote-unlikely-before-2026-session'
ground_truth = article_ground_truth(mpr_link,0,0,0,0,ground_truth)

In [ ]:
huff_post = 'https://www.huffpost.com/entry/gretchen-carlson-trump-quiet-piggy-attack_n_691d01f7e4b06f2a60ca504b'
ground_truth = article_ground_truth(huff_post,0,0,0,0,ground_truth)

In [ ]:
scripps_link = 'https://www.scrippsnews.com/politics/immigration/california-revokes-17-000-commercial-drivers-licenses-for-immigrants'
ground_truth = article_ground_truth(scripps_link,1,0,1,1,ground_truth)

In [ ]:
texastribune_link = 'https://www.texastribune.org/2025/10/14/police-dallas-hero-crime-proposition/'
ground_truths = article_ground_truth(texastribune_link,1,0,0,1,ground_truth)

In [ ]:
ground_truth

,statement,frequency_heuristic,malicious_account,sensationalism,naive_realism,link
0,A president steeped in constitutional lore mig...,2,0,2,2,https://www.cnn.com/2025/10/20/politics/trump-...
1,"Trump is 'committed' to $2,000 tariff dividend...",0,0,0,1,https://abc7.com/post/trump-is-committed-givin...
2,The longest US government shutdown in history ...,0,0,0,1,https://www.theguardian.com/us-news/2025/nov/1...
3,Add AP News as your preferred source to see mo...,0,0,0,0,https://apnews.com/article/redistricting-gerry...
4,"Key Points Prop 50 passed in California, appro...",1,0,1,0,https://azcapitoltimes.com/news/2025/11/12/pro...
5,"The bill will now head to the Senate, where it...",0,0,1,1,https://abcnews.go.com/Politics/house-vote-ful...
6,More than 6 in 10 registered voters said they ...,0,0,0,0,https://www.nbcnews.com/politics/politics-news...
7,"UK will not tolerate Chinese spying, minister ...",1,0,0,0,https://www.bbc.com/news/articles/c4gpnz05kr8o
8,Most Californians probably see the Capitol as ...,0,0,1,1,https://calmatters.org/commentary/2025/11/dana...
9,A new KFF/New York Times Survey of Immigrants ...,0,0,2,0,https://www.kff.org/racial-equity-and-health-p...
